In [1]:
import pandas as pd
import os

df = pd.DataFrame()
"""
for json_file in os.listdir("dataset/jsonl/"):
    df = pd.concat([df, pd.read_json("dataset/jsonl/" + json_file, lines=True)])
    print(f"Added {json_file} to dataframe.")
"""
df = pd.read_json('dataset/jsonl/en-US.jsonl', lines=True)
df["locale"] = df["locale"].apply(lambda x: x.split("-")[0])

In [2]:
%run -i "preprocess_lang.ipynb"
%run -i "preprocess_nolang.ipynb"
%run -i "evaluation.ipynb"

In [3]:
from toolz.functoolz import pipe

params = [
    remove_punctuation,
    lowercase,
    tokenize,
    apply_stemming,
    encode_labels
]

df = pipe(
    df,
    *params
)


In [4]:
df['utt'] = df['utt'].apply(lambda x: " ".join(x))

print(f"Finished preprocessing dataset.\n\n")

Finished preprocessing dataset.




In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df['utt'].to_list())


In [6]:
input_cols = [f'__{feature_name}' for feature_name in vectorizer.get_feature_names_out()]
output_cols = ['intent']
averages = [None, "macro", "weighted", "micro", "samples"]

In [7]:
count_vect_df = pd.DataFrame(vectorized.todense(), columns=input_cols)
df = pd.concat([df, count_vect_df], axis=1)

In [8]:
from sklearn.naive_bayes import MultinomialNB

nb = train_and_use_model(
    MultinomialNB(),
    {
        "model__alpha": [0.1, 0.5, 1.0, 1.5, 2.0],
    },
    n_iter=5,
)

C:\Users\nrtc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best params for MultinomialNB: {'model__alpha': 0.1}
Evaluation metrics for RandomizedSearchCV
RandomizedSearchCV's default score metric: 0.7541365405179443
Classification report
              precision    recall  f1-score   support

           0     0.8158    0.9118    0.8611        34
           1     0.8889    0.7619    0.8205        21
           2     0.8718    0.8293    0.8500        41
           3     1.0000    0.8182    0.9000        11
           4     0.8529    0.9062    0.8788        32
           5     1.0000    0.1667    0.2857         6
           6     0.6111    0.8462    0.7097        13
           7     0.7226    0.7857    0.7529       126
           8     0.8354    0.9851    0.9041        67
           9     0.8638    0.8804    0.8720       209
          11     0.8986    0.8611    0.8794        72
          12     0.8000    0.8000    0.8000        15
          13     0.8409    0.8409    0.8409        88
          14     0.5455    0.5000    0.5217        12
          

# MultinomialNB

Best params:
{
    'model__alpha': 0.1
}

In [9]:
#import xgboost as xgb
import xgboost as xgb

xgboost = train_and_use_model(
    xgb.XGBClassifier(),
    {
        "model__n_estimators": [100, 200, 300, 400, 500],
        "model__max_depth": [3, 4, 5, 6, 7, 8, 9, 10],
        "model__learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
        "model__booster": ["gbtree", "gblinear", "dart"],
        "model__gamma": [0, 0.25, 0.5, 1.0],
        "model__min_child_weight": [1, 3, 5, 7],
        "model__max_delta_step": [0, 0.2, 0.6, 1.0],
        "model__subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    },
    n_iter=3
)

C:\Users\nrtc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
9 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nrtc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nrtc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\

[04:44:03] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "gamma", "max_delta_step", "max_depth", "min_child_weight", "subsample" } are not used.

Best params for XGBClassifier: {'model__subsample': 0.7, 'model__n_estimators': 200, 'model__min_child_weight': 3, 'model__max_depth': 3, 'model__max_delta_step': 1.0, 'model__learning_rate': 0.05, 'model__gamma': 0.5, 'model__booster': 'gblinear'}
Evaluation metrics for RandomizedSearchCV
RandomizedSearchCV's default score metric: 0.7887292811188353
Classification report
              precision    recall  f1-score   support

           0     0.9032    0.8235    0.8615        34
           1     0.9474    0.8571    0.9000        21
           2     0.7907    0.8293    0.8095        41
           3     0.8182    0.8182    0.8182        11
           4     0.8333    0.9375    0.8824        32
           5     1.0000    0.3333    0.50

# XGBoost

Best Params:
{
    'model__subsample': 0.7,
    'model__n_estimators': 200, 
    'model__min_child_weight': 3, 
    'model__max_depth': 3, 
    'model__max_delta_step': 1.0, 
    'model__learning_rate': 0.05, 
    'model__gamma': 0.5, 
    'model__booster': 'gblinear'
}

In [10]:
#import knn
from sklearn.neighbors import KNeighborsClassifier

knn = train_and_use_model(
    KNeighborsClassifier(),
    {
        "model__n_neighbors": [3, 5, 11, 19],
        "model__weights": ["uniform", "distance"],
        "model__algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
        "model__leaf_size": [30, 40, 50, 60, 70, 80, 90, 100],
        "model__p": [1, 2],
        "model__metric": ["minkowski", "euclidean", "manhattan"],
    },
    n_iter=50
)

In [ ]:
"""df = pd.DataFrame()

for json_file in os.listdir("dataset/jsonl/"):
    df = pd.concat([df, pd.read_json("dataset/jsonl/" + json_file, lines=True)])
    print(f"Added {json_file} to dataframe.")

df["locale"] = df["locale"].apply(lambda x: x.split("-")[0])"""

In [ ]:
"""from toolz.functoolz import pipe

params = [
    remove_punctuation,
    lowercase,
    tokenize,
    apply_stemming,
    encode_labels
]

df = pipe(
    df,
    *params
)

df['utt'] = df['utt'].apply(lambda x: " ".join(x))

print(f"Finished preprocessing dataset.\n\n")

from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = TfidfTransformer()
vectorized = vectorizer.fit_transform(df['utt'].to_list())
input_cols = [f'__{feature_name}' for feature_name in vectorizer.get_feature_names_out()]
output_cols = ['intent']
averages = [None, "macro", "weighted", "micro", "samples"]"""

In [ ]:
"""training_inputs = df[df['partition'] != 'test'][input_cols].values
testing_inputs = df[df['partition'] == 'test'][input_cols].values
training_classes = df[df['partition'] != 'test'][output_cols].values
testing_classes = df[df['partition'] == 'test'][output_cols].values"""

In [ ]:
"""from sklearn.metrics import f1_score

nb.fit(training_inputs, training_classes)
score = nb.score(testing_inputs, testing_classes)
f1_score = f1_score(testing_classes, nb.predict(testing_inputs), average="macro")

print(f"Score: {score}")
print(f"F1 Score: {f1_score}")"""

In [ ]:
"""xgboost.fit(training_inputs, training_classes)
score = xgboost.score(testing_inputs, testing_classes)
f1_score = f1_score(testing_classes, xgboost.predict(testing_inputs), average="macro")

print(f"Score: {score}")
print(f"F1 Score: {f1_score}")"""

In [ ]:
"""knn.fit(training_inputs, training_classes)
score = knn.score(testing_inputs, testing_classes)
f1_score = f1_score(testing_classes, knn.predict(testing_inputs), average="macro")

print(f"Score: {score}")
print(f"F1 Score: {f1_score}")"""